In [350]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

In [351]:
feature_df = pd.read_csv("Features.csv")
advanced_feature_df = pd.read_csv("Advanced_Features.csv")
statistical_feature_df = pd.read_csv("Statistical_Features.csv")

feature_df = pd.merge(feature_df, advanced_feature_df, left_index=True, right_index=True)
feature_df = pd.merge(feature_df, statistical_feature_df, left_index=True, right_index=True)

feature_df.drop(feature_df.columns[feature_df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

labels = feature_df["Label_y"]
feature_df["Label"] = labels
feature_df = feature_df.drop(columns = ["Label_y", "Label_x"])

In [352]:
# Labels are the values we want to predict
labels = np.array(feature_df['Label'])
# Remove the labels from the features
# axis 1 refers to the columns
features = feature_df.drop('Label', axis = 1)

In [353]:
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

In [354]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.9)

In [355]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [356]:
y_pred = rf.predict(X_test)

In [357]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9986111111111111


This is the accuracy for classifying NREM sleep and Awake get with a simple classifier, using 3 basic topological summary statistics and Adcock Carlson coordinates, as well as statistical features.